# Chat completions

In [2]:
# OpenAI client configuration
import os
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("API_KEY")

base_url = "https://albert.api.dev.etalab.gouv.fr/v1"
api_key = "albert-vdyqqiSdmWrvYJCVw9rLgR5WJLo72yKqxQyXe5zZTjWEMtKH2VZTuL7XNqwhsLVN"

client = OpenAI(base_url=base_url, api_key=api_key)

In [3]:
# unstreamed chat
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Salut Albert !"}],
    "stream": False,
    "n": 1,
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Bonjour ! Je m'appelle Albert, mais je suis un modèle de langage artificiel, donc je ne suis pas vraiment Albert, mais je suis là pour discuter avec toi ! Comment ça va ?


In [4]:
# streamed chat
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Salut Albert !"}],
    "stream": True,
    "n": 1,
}

response = client.chat.completions.create(**data)
for chunk in response:
    if chunk.choices[0].finish_reason is not None:
        break
    print(chunk.choices[0].delta.content, end='', flush=True)


NoneSalut ! Je suis Albert, un modèle de langage artificiel. Je suis ravi de vous voir ! Qu'est-ce que vous voulez discuter ?